# Test CMIP6



In [1]:
import xarray as xr
from clisops.core import subset
from clisops.ops import subset as subset_op
from clisops.utils.dataset_utils import open_xr_dataset

import time
import os

In [2]:
basedir_cmip6 = "/mnt/lustre/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-ESM1-2-HR/historical/r1i1p1f1"

cmip6_nc = f"{basedir_cmip6}/day/ta/gn/v20190710/ta_day_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_20100101-20141231.nc"
cmip6_nc

'/mnt/lustre/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-ESM1-2-HR/historical/r1i1p1f1/day/ta/gn/v20190710/ta_day_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_20100101-20141231.nc'

## ncdump - CMIP6

In [3]:
! du -sh {cmip6_nc}

1.6G	/mnt/lustre/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-ESM1-2-HR/historical/r1i1p1f1/day/ta/gn/v20190710/ta_day_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_20100101-20141231.nc


In [4]:
! ncdump -sh {cmip6_nc}

netcdf ta_day_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_20100101-20141231 {
dimensions:
	time = UNLIMITED ; // (1826 currently)
	plev = 8 ;
	lat = 192 ;
	lon = 384 ;
	bnds = 2 ;
variables:
	double time(time) ;
		time:bounds = "time_bnds" ;
		time:units = "days since 1850-1-1 00:00:00" ;
		time:calendar = "proleptic_gregorian" ;
		time:axis = "T" ;
		time:long_name = "time" ;
		time:standard_name = "time" ;
		time:_Storage = "chunked" ;
		time:_ChunkSizes = 1 ;
		time:_Endianness = "little" ;
	double time_bnds(time, bnds) ;
		time_bnds:_Storage = "chunked" ;
		time_bnds:_ChunkSizes = 1, 2 ;
		time_bnds:_Shuffle = "true" ;
		time_bnds:_DeflateLevel = 1 ;
		time_bnds:_Endianness = "little" ;
	double plev(plev) ;
		plev:units = "Pa" ;
		plev:axis = "Z" ;
		plev:positive = "down" ;
		plev:long_name = "pressure" ;
		plev:standard_name = "air_pressure" ;
		plev:_Storage = "contiguous" ;
		plev:_Endianness = "little" ;
	double lat(lat) ;
		lat:bounds = "lat_bnds" ;
		lat:units = "degrees_north" ;
	

## helper functions

In [5]:
def ds_info(ds):
    # compression levels
    print("data vars with compression:")
    for var in ds.data_vars:
        # print(var, ds[var].encoding, "\n")
        complevel = ds[var].encoding.get("complevel", 0)
        print(var, "compression level =", complevel)
        
    # fill values
    print("\nfill values:")
    var_list = list(ds.coords) + list(ds.data_vars)
    for var in var_list:
        fill_value = ds[var].encoding.get("_FillValue")
        print(var, "fill value =", fill_value)
        
    # string attributes with compression
    print("\nstring attributes with compression:")
    for cvar in [
            "member_id",
            "gcm_variant",
            "gcm_model",
            "gcm_institution",
            "rcm_variant",
            "rcm_model",
            "rcm_institution",
        ]:
            for en in ["zlib", "shuffle", "complevel"]:
                try:
                    print(cvar, en, ds[cvar].encoding[en])
                except KeyError:
                    pass

## xarray - CMIP6



In [6]:
ds_cmip6 = xr.open_dataset(cmip6_nc)
ds_cmip6

<xarray.Dataset>
Dimensions:    (time: 1826, bnds: 2, plev: 8, lat: 192, lon: 384)
Coordinates:
  * time       (time) datetime64[ns] 2010-01-01T12:00:00 ... 2014-12-31T12:00:00
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat        (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * lon        (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    ta         (time, plev, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            ta
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/43b8c97b-1c79-43f9-b276-49d36047fbb2

In [7]:
ds_info(ds_cmip6)

data vars with compression:
time_bnds compression level = 1
lat_bnds compression level = 1
lon_bnds compression level = 1
ta compression level = 1

fill values:
time fill value = None
plev fill value = None
lat fill value = None
lon fill value = None
time_bnds fill value = None
lat_bnds fill value = None
lon_bnds fill value = None
ta fill value = 1e+20

string attributes with compression:


In [8]:
ds = ds_cmip6.isel(time=slice(0, 12), lon=slice(30, 50), lat=slice(50, 70))
ds

<xarray.Dataset>
Dimensions:    (time: 12, bnds: 2, plev: 8, lat: 20, lon: 20)
Coordinates:
  * time       (time) datetime64[ns] 2010-01-01T12:00:00 ... 2010-01-12T12:00:00
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat        (lat) float64 -42.55 -41.61 -40.68 ... -26.65 -25.71 -24.78
  * lon        (lon) float64 28.12 29.06 30.0 30.94 ... 43.12 44.06 45.0 45.94
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    ta         (time, plev, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            ta
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/43b8c97b-1c79-43f9-b276-49d36047fbb2

In [9]:
# clean up outputs

! rm /tmp/output_*

In [10]:
ds.to_netcdf("/tmp/output_cmip6_xarray.nc")


## clisops-core - cmip6



In [11]:
ds_cmip6 = open_xr_dataset(cmip6_nc)
ds_cmip6

<xarray.Dataset>
Dimensions:    (time: 1826, bnds: 2, plev: 8, lat: 192, lon: 384)
Coordinates:
  * time       (time) object 2010-01-01 12:00:00 ... 2014-12-31 12:00:00
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat        (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * lon        (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    ta         (time, plev, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            ta
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/43b8c97b-1c79-43f9-b276-49d36047fbb2

In [12]:
ds = subset.subset_bbox(
    ds_cmip6, lat_bnds=[45, 50], lon_bnds=[-60, -55],
    start_date='2013-01-01', end_date='2013-01-30')
ds

subset_time, start_date=2013-01-01, end_date=2013-01-30
check_start_end_dates {'start_date': '2013-01-01', 'end_date': '2013-01-30'}
check2
check3
subset_time2 2013-01-01 2013-01-30


<xarray.Dataset>
Dimensions:    (time: 30, bnds: 2, plev: 8, lat: 5, lon: 6)
Coordinates:
  * time       (time) object 2013-01-01 12:00:00 ... 2013-01-30 12:00:00
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat        (lat) float64 45.35 46.29 47.22 48.16 49.09
  * lon        (lon) float64 300.0 300.9 301.9 302.8 303.8 304.7
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    ta         (time, plev, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            ta
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/43b8c97b-1c79-43f9-b276-49d36047fbb2

In [13]:
ds.to_netcdf("/tmp/output_cmip6_clisops_core.nc")


## clisops-ops - cmip6 - subset

TODO: it takes too long ... 21 secs (when used together with bbox)





In [14]:
# clean up outputs

! rm /tmp/output_*

In [15]:
ds_cmip6 = open_xr_dataset(cmip6_nc)
ds_cmip6

<xarray.Dataset>
Dimensions:    (time: 1826, bnds: 2, plev: 8, lat: 192, lon: 384)
Coordinates:
  * time       (time) object 2010-01-01 12:00:00 ... 2014-12-31 12:00:00
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat        (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * lon        (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    ta         (time, plev, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            ta
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/43b8c97b-1c79-43f9-b276-49d36047fbb2

In [16]:
start = time.time()

outputs = subset_op(
    ds=ds_cmip6,
    time="2013-01/2013-01",
    # time_components="year:2013",
    area=(0.0, 49.0, 10.0, 65.0),
    output_type="nc",
    # output_type="xarray",
    output_dir="/tmp",
    split_method="time:auto",
    file_namer="simple"
)

duration = time.time() - start
print(f"duration: {duration} secs")

print("Number of output files: ", len(outputs))
outputs[0]

calculate {'lon_bnds': (0.0, 10.0), 'lat_bnds': (49.0, 65.0), 'start_date': '2013-01-01T00:00:00', 'end_date': '2013-01-31T23:59:59'}
subset_bbox {'lon_bnds': (0.0, 10.0), 'lat_bnds': (49.0, 65.0), 'start_date': '2013-01-01T00:00:00', 'end_date': '2013-01-31T23:59:59', 'first_level': None, 'last_level': None, 'time_values': None, 'level_values': None}
subset_time, start_date=2013-01-01T00:00:00, end_date=2013-01-31T23:59:59
check_start_end_dates {'start_date': '2013-01-01T00:00:00', 'end_date': '2013-01-31T23:59:59'}
check2
check2.1
check2.3
check3
check3.1
check3.3
subset_time2 2013-01-01T12:00:00 2013-01-31T12:00:00
result [cftime.DatetimeProlepticGregorian(2013, 1, 1, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeProlepticGregorian(2013, 1, 2, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeProlepticGregorian(2013, 1, 3, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeProlepticGregorian(2013, 1, 4, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeProlepticGregorian(2013, 1, 5, 

'/tmp/output_001.nc'

In [17]:
file_size = os.path.getsize(outputs[0])
print("File Size is :", file_size/(1024*1024), "MB")

File Size is : 0.14179420471191406 MB


In [18]:
ds = xr.open_dataset(outputs[0])
ds

<xarray.Dataset>
Dimensions:    (time: 31, bnds: 2, plev: 8, lat: 18, lon: 11)
Coordinates:
  * time       (time) datetime64[ns] 2013-01-01T12:00:00 ... 2013-01-31T12:00:00
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat        (lat) float64 49.09 50.03 50.96 51.9 ... 62.18 63.12 64.05 64.99
  * lon        (lon) float64 0.0 0.9375 1.875 2.812 ... 6.562 7.5 8.438 9.375
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    ta         (time, plev, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            ta
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/43b8c97b-1c79-43f9-b276-49d36047fbb2

In [19]:
ds_info(ds)

data vars with compression:
time_bnds compression level = 1
lat_bnds compression level = 1
lon_bnds compression level = 1
ta compression level = 1

fill values:
time fill value = None
plev fill value = None
lat fill value = None
lon fill value = None
time_bnds fill value = None
lat_bnds fill value = None
lon_bnds fill value = None
ta fill value = 1e+20

string attributes with compression:


## clisops-ops - cmip6 - subset with patch

TODO: the subsetting takes too long ... here is a patch to fix this.

In [20]:
# monkey patch for clisops

from clisops.utils import dataset_utils
import cftime

def custom_is_time(coord):
    print(f"Custom behavior for is_time with input: {coord.name}")
    
    import numpy as np
    
    if "time" in coord.cf.coordinates and coord.name in coord.cf.coordinates["time"]:
        return True

    if (
        "time" in coord.cf.standard_names
        and coord.name in coord.cf.standard_names["time"]
    ):
        return True

    if np.issubdtype(coord.dtype, np.datetime64):
        return True

    # TODO: this code leads to memory overflow when applied on a data variable!
    print(f"skip np.atleast_1d(coord.values) on: {coord.name}")
    # _check_coord = True
    # if _check_coord and isinstance(np.atleast_1d(coord.values)[0], cftime.datetime):
    # value = next(iter(coord.values))
    if False:
        value = coord.values[0]
        if isinstance(value, cftime.datetime):
            return True

    if hasattr(coord, "axis"):
        if coord.axis == "T":
            return True

    return False


# Monkey patch the function
dataset_utils.is_time = custom_is_time


In [21]:
# clean up outputs

! rm /tmp/output_*

In [22]:
ds_cmip6 = open_xr_dataset(cmip6_nc)
ds_cmip6

<xarray.Dataset>
Dimensions:    (time: 1826, bnds: 2, plev: 8, lat: 192, lon: 384)
Coordinates:
  * time       (time) object 2010-01-01 12:00:00 ... 2014-12-31 12:00:00
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat        (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * lon        (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    ta         (time, plev, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            ta
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/43b8c97b-1c79-43f9-b276-49d36047fbb2

In [23]:
start = time.time()

outputs = subset_op(
    ds=ds_cmip6,
    time="2013-01/2013-01",
    area=(0.0, 50.0, 10.0, 60.0),
    output_type="nc",
    # output_type="xarray",
    output_dir="/tmp",
    split_method="time:auto",
    file_namer="simple"
)

duration = time.time() - start
print(f"duration: {duration} secs")

outputs[0]

calculate {'lon_bnds': (0.0, 10.0), 'lat_bnds': (50.0, 60.0), 'start_date': '2013-01-01T00:00:00', 'end_date': '2013-01-31T23:59:59'}
Custom behavior for is_time with input: time
Custom behavior for is_time with input: time_bnds
skip np.atleast_1d(coord.values) on: time_bnds
Custom behavior for is_time with input: lat_bnds
skip np.atleast_1d(coord.values) on: lat_bnds
Custom behavior for is_time with input: lon_bnds
skip np.atleast_1d(coord.values) on: lon_bnds
Custom behavior for is_time with input: ta
skip np.atleast_1d(coord.values) on: ta
Custom behavior for is_time with input: time
Custom behavior for is_time with input: time_bnds
skip np.atleast_1d(coord.values) on: time_bnds
Custom behavior for is_time with input: lat_bnds
skip np.atleast_1d(coord.values) on: lat_bnds
Custom behavior for is_time with input: lon_bnds
skip np.atleast_1d(coord.values) on: lon_bnds
Custom behavior for is_time with input: ta
skip np.atleast_1d(coord.values) on: ta
subset_bbox {'lon_bnds': (0.0, 10.0)

'/tmp/output_001.nc'

In [24]:
file_size = os.path.getsize(outputs[0])
print("File Size is :", file_size/(1024*1024), "MB")

File Size is : 0.10758399963378906 MB


In [25]:
ds = xr.open_dataset(outputs[0])
ds

<xarray.Dataset>
Dimensions:    (time: 31, bnds: 2, plev: 8, lat: 11, lon: 11)
Coordinates:
  * time       (time) datetime64[ns] 2013-01-01T12:00:00 ... 2013-01-31T12:00:00
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat        (lat) float64 50.03 50.96 51.9 52.83 ... 56.57 57.51 58.44 59.38
  * lon        (lon) float64 0.0 0.9375 1.875 2.812 ... 6.562 7.5 8.438 9.375
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    ta         (time, plev, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            ta
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/43b8c97b-1c79-43f9-b276-49d36047fbb2

## check numpy function

this code line makes troubles:

#if isinstance(np.atleast_1d(coord.values)[0], cftime.datetime):

In [26]:
import numpy as np

In [27]:
ds = open_xr_dataset(cmip6_nc)
ds

<xarray.Dataset>
Dimensions:    (time: 1826, bnds: 2, plev: 8, lat: 192, lon: 384)
Coordinates:
  * time       (time) object 2010-01-01 12:00:00 ... 2014-12-31 12:00:00
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat        (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * lon        (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    ta         (time, plev, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            ta
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/43b8c97b-1c79-43f9-b276-49d36047fbb2

### check time coord

In [28]:
time_coord = ds.coords["time"]
time_coord

<xarray.DataArray 'time' (time: 1826)>
array([cftime.DatetimeProlepticGregorian(2010, 1, 1, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeProlepticGregorian(2010, 1, 2, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeProlepticGregorian(2010, 1, 3, 12, 0, 0, 0, has_year_zero=True),
       ...,
       cftime.DatetimeProlepticGregorian(2014, 12, 29, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeProlepticGregorian(2014, 12, 30, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeProlepticGregorian(2014, 12, 31, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 2010-01-01 12:00:00 ... 2014-12-31 12:00:00
Attributes:
    bounds:         time_bnds
    axis:           T
    long_name:      time
    standard_name:  time

In [29]:
time_coord.name

'time'

In [30]:
len(time_coord.values)

1826

In [31]:
np.atleast_1d(time_coord.values)[0]

cftime.DatetimeProlepticGregorian(2010, 1, 1, 12, 0, 0, 0, has_year_zero=True)

In [32]:
np.atleast_1d?

Signature:       np.atleast_1d(*arys)
Call signature:  np.atleast_1d(*args, **kwargs)
Type:            _ArrayFunctionDispatcher
String form:     <function atleast_1d at 0x7f96729753a0>
File:            ~/.envs/rook/lib/python3.11/site-packages/numpy/core/shape_base.py
Docstring:      
Convert inputs to arrays with at least one dimension.

Scalar inputs are converted to 1-dimensional arrays, whilst
higher-dimensional inputs are preserved.

Parameters
----------
arys1, arys2, ... : array_like
    One or more input arrays.

Returns
-------
ret : ndarray
    An array, or list of arrays, each with ``a.ndim >= 1``.
    Copies are made only if necessary.

See Also
--------
atleast_2d, atleast_3d

Examples
--------
>>> np.atleast_1d(1.0)
array([1.])

>>> x = np.arange(9.0).reshape(3,3)
>>> np.atleast_1d(x)
array([[0., 1., 2.],
       [3., 4., 5.],
       [6., 7., 8.]])
>>> np.atleast_1d(x) is x
True

>>> np.atleast_1d(1, [3, 4])
[array([1]), array([3, 4])]
Class docstring:
Class to wrap functi

In [33]:
np.atleast_1d(1.0)

array([1.])

In [34]:
np.atleast_1d([1, 2, 3])

array([1, 2, 3])

In [35]:
np.atleast_1d([[1, 2 , 3], [4, 5, 6]])

array([[1, 2, 3],
       [4, 5, 6]])

### check also ta data variable

This will explode the memory ... in this case "only" up to 9 GB

In [36]:
start = time.time()

print("numer of values", len(ds.ta.values))

duration = time.time() - start
print(f"duration: {duration} secs")

numer of values 1826
duration: 21.469820499420166 secs


In [37]:
np.atleast_1d(ds.ta.values)[0]

array([[[260.1153 , 260.10944, 260.10553, ..., 260.13287, 260.127  ,
         260.12115],
        [259.75983, 259.72467, 259.69147, ..., 259.8614 , 259.8282 ,
         259.79303],
        [260.09772, 260.05865, 260.0196 , ..., 260.2149 , 260.17584,
         260.13678],
        ...,
        [248.84186, 249.04108, 249.26959, ..., 248.43756, 248.53717,
         248.67194],
        [250.836  , 250.88483, 250.94147, ..., 250.74225, 250.76569,
         250.79694],
        [251.05865, 251.08014, 251.10358, ..., 250.99811, 251.01764,
         251.03717]],

       [[252.19507, 252.19116, 252.1853 , ..., 252.21265, 252.20679,
         252.20093],
        [251.84937, 251.81616, 251.78296, ..., 251.94897, 251.91577,
         251.88257],
        [252.17749, 252.14038, 252.10327, ..., 252.29077, 252.25366,
         252.2146 ],
        ...,
        [241.81812, 241.9685 , 242.12476, ..., 241.37866, 241.5232 ,
         241.66772],
        [244.10132, 244.19702, 244.29468, ..., 243.82593, 243.91577,
   

In [38]:
ds.dims

Frozen({'time': 1826, 'bnds': 2, 'plev': 8, 'lat': 192, 'lon': 384})

In [39]:
ds.ta.cf.coordinates

{'longitude': ['lon'],
 'latitude': ['lat'],
 'vertical': ['plev'],
 'time': ['time']}

In [40]:
ds.ta.dims

('time', 'plev', 'lat', 'lon')

In [41]:
ds.coords["time"].dims

('time',)